In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
import gc
import sys
import time
import feather
import numpy as np
import pandas as pd
from contextlib import contextmanager
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib
import optuna

sys.path.append("/Users/ueda/Desktop/kaggle/mynavi/code/src/")
from logger import setup_logger, LOGGER
from trainer import train_lgbm
from util_tool import reduce_mem_usage

In [2]:
# ===============
# Constants
# ===============
DATA_DIR = "../input/"
TRAIN_PATH = os.path.join(DATA_DIR, "train.csv")
TEST_PATH = os.path.join(DATA_DIR, "test.csv")
#META_PATH = os.path.join(DATA_DIR, "meta.csv")
#META_PATH2 = os.path.join(DATA_DIR, "minincome_clean.csv")
SUB_PATH = os.path.join(DATA_DIR, "sample_submit.csv")
FOLDS_PATH = "../input/mynavi_Stratifiedfold01.feather"  #folds気をつけて
LOGGER_PATH = "log.txt"
ID_COLUMN = "id"


EXP_ID = "exp0_morimori_feature_reduce"

SAVE_PATH = f'../output/{EXP_ID}'
try:
    os.mkdir(SAVE_PATH)
except:
    print("Already directory created")

Already directory created


In [9]:
# ===============
# Settings
# ===============
pd.set_option('display.max_columns', 50)

SEED = 0
N_SPLITS = 5
LOG = False
LGBM_PARAMS = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'poisson',    #regression, gamma, poisson gamma早すぎ
    'metric': "rmse",
    'learning_rate': 0.1,
    'num_leaves': 15,
    'subsample': 0.9,
    'subsample_freq': 1,
    'colsample_bytree': 0.8,
    'max_depth': 9,
    'max_bin': 255,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'min_child_samples': 20,
    'min_gain_to_split': 0.02,
    'min_data_in_bin': 3,
    'bin_construct_sample_cnt': 5000,
    'cat_l2': 5,
    'verbose': -1,
    'nthread': -1,
    'seed': SEED,
}
LGBM_FIT_PARAMS = {
    'num_boost_round': 50000,
    'early_stopping_rounds': 100,
    'verbose_eval': 5000,
}

use_cols = []

drop_columns = ['fold_id', 'id', 'キッチン', 'バス・トイレ', '周辺環境',
                '室内設備', '所在地', '所在階', '放送・通信', '間取り',
                '築年数', '賃料', '面積', '駐車場', "路線_0", "駅_0", "方角",
                 "路線_1", "駅_1", "路線_2", "駅_2", "契約期間", '建物構造','アクセス', "地域名", "地域_n丁目"
]

categorical_features = ['所在_区', "catuse_駅_0", "catuse_路線_0",
                        "cat_int間取り", "cat_building_height" #, '建物構造'
]


TE_columns = [#"所在_区", #'catuse_路線_0', 'catuse_駅_0', 
              #"所在_区", #"間取りtype", #'catuse_路線_0', 'catuse_駅_0', "int築年"
              #"cat_int間取り", #"cat_building_height" # "定期借家", '一戸建て'
]

FEATURES = [
    "../code/feature_csv/base_feature1.feather",
    #"../code/feature_csv/base_group_feature2.feather",
    "../code/feature_csv/nonleak_group_feature2.feather",
    #'../code/feature_csv/nonleak_nearStation_group_feature.feather',
    #'../code/feature_csv/near_access_feature.feather',
    "../code/feature_csv/word_contain_sparse_feature.feather",
    "../code/feature_csv/access_feature.feather"
]
setup_logger(out_file=LOGGER_PATH)

2019-11-04 15:35:24,706 - INFO - logger set up


<RootLogger root (DEBUG)>

In [15]:
# ===============
# Functions
# ===============
@contextmanager
def timer(name):
    t0 = time.time()
    yield
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s')
    
def load_feature_feather(df, filename):
    feature = feather.read_dataframe(filename)
    df = pd.concat((df, feature), axis=1)
    del feature
    gc.collect()
    return df

def load_feature_csv(df, filename):
    feature = pd.csv(filename)
    df = pd.concat((df, feature), axis=1)
    del feature
    gc.collect()
    return df

def calc_loss(y_true, y_pred):
    return mean_squared_error(y_true, y_pred)

## main

In [20]:
with timer("load data"):
    train = feather.read_dataframe(FOLDS_PATH)
    test = pd.read_csv(TEST_PATH)
    if LOG:
        y = np.log1p(train['賃料'].copy())
    else:
        y = train['賃料'].copy()
    folds = train[["fold_id"]].copy()
    n_train = len(train)
    train = train.append(test).reset_index(drop=True)
    del test
    gc.collect()

2019-11-04 15:37:44,365 - INFO - [load data] done in 1 s


In [21]:
with timer("load featuers"):
    #train = merge_meta(train, META_PATH, META_PATH2)
    for f in FEATURES:
        train = load_feature_feather(train, f)

2019-11-04 15:37:45,778 - INFO - [load featuers] done in 1 s


In [22]:
with timer("preprocessing"):
    for c in categorical_features:
        lbl = LabelEncoder()
        lbl.fit(list(train[c].astype("str").values))
        train[c] = lbl.transform(list(train[c].astype("str").values))
        
    """importtanceで区切るなら"""
    if len(use_cols)==0:
        test = train[n_train:]
        train = train[:n_train]
    
    elif len(use_cols)>0:
        test = test[use_cols]
        train = train[use_cols]
    
    try:
        train.drop(drop_columns, axis=1, inplace=True)
        test.drop(drop_columns, axis=1, inplace=True)
        print("drop done")
    except:
        pass
    
    train = reduce_mem_usage(train)
    train.to_feather(f"../features/{EXP_ID}_df.feather")
    
        
    features = list(train.columns.values)
    
    #gc.collect()
    
    #categorical_features = [cat_col for cat_col in categorical_features if cat_col not in TE_columns]
    
    target_test = pd.read_csv(f'../code/feature_csv/target_encoding_test.csv')
    test[TE_columns] = target_test[TE_columns]

drop done
Memory usage of dataframe is 111.89 MB
column =  466
0
50
100
150
200
250
300
350
400
450
Memory usage after optimization is: 40.73 MB
Decreased by 63.6%


2019-11-04 15:37:51,980 - INFO - [preprocessing] done in 6 s


In [ ]:
with timer("train"):
    y_oof = np.empty(len(train), )
    y_test = []
    feature_importances = pd.DataFrame()

    for fold_id in range(N_SPLITS):
        with timer(f"fold {fold_id}"):
            x_train, y_train = train[folds.fold_id != fold_id], y[folds.fold_id != fold_id]
            x_val, y_val = train[folds.fold_id == fold_id], y[folds.fold_id == fold_id]
            
            #TE_feature = pd.read_csv(f'../code/feature_csv/target_encoding_groupfold{fold_id}.csv')
            #x_train[TE_columns] = TE_feature[TE_feature["fold_id"]!=fold_id][TE_columns]
            #x_val[TE_columns] = TE_feature[TE_feature["fold_id"]==fold_id][TE_columns]

            y_pred_valid, y_pred_test, train_loss, valid_loss, importances, best_iter = train_lgbm(
                x_train, y_train, x_val, y_val, test,
                categorical_features=categorical_features,
                feature_name=features,
                fold_id=fold_id,
                lgb_params=LGBM_PARAMS,
                fit_params=LGBM_FIT_PARAMS,
                model_name=EXP_ID,
                loss_func=calc_loss,
                rank=False,
                calc_importances=True
            )
            y_oof[folds.fold_id == fold_id] = y_pred_valid
            y_test.append(y_pred_test)
            feature_importances = pd.concat([feature_importances, importances], axis=0, sort=False)

    feature_importances.to_csv(os.path.join(SAVE_PATH, "feature_importances.csv"), index=False)
    if LOG:
        y = np.expm1(y)
        y_oof = np.expm1(y_oof)
        y_test = np.expm1(y_test)
    score = calc_loss(y, y_oof)
    y_test = np.mean(y_test, axis=0)
    #np.save(os.path.join(SAVE_PATH, "oof.npy"), y_oof)
    #np.save(os.path.join(SAVE_PATH, "y.npy"), y)
    yoof_and_target = pd.DataFrame()
    yoof_and_target["賃料"] = y
    yoof_and_target[f'{EXP_ID}'] = y_oof
    yoof_and_target.to_csv(os.path.join(SAVE_PATH, "yoof_and_target.csv"), index=False)
    yoof_and_target.to_csv(f'../output/vs_trains/{EXP_ID}.csv', index=False)
    
    test_result = pd.DataFrame()
    test_result[f"{EXP_ID}"] = y_test
    test_result.to_csv(f'../output/test_results/{EXP_ID}.csv', index=False)
    
    LOGGER.info(f'CV={score}')

Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[3078]	valid's rmse: 17373.2


2019-11-04 15:38:15,265 - INFO - Best Iteration: 3078
2019-11-04 15:38:18,283 - INFO - [fold 0] done in 26 s


Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[2736]	valid's rmse: 20792.9


2019-11-04 15:38:41,651 - INFO - Best Iteration: 2736
2019-11-04 15:38:44,453 - INFO - [fold 1] done in 26 s


Training until validation scores don't improve for 100 rounds.


In [ ]:
"""if fold_id does not drop """
#(x_train[x_train["fold_id"]!=0]["fold_id"] == TE_feature[TE_feature["fold_id"]!=0]["fold_id"]).value_counts()

In [ ]:
#TE_feature[TE_columns]["所在_区"].value_counts()

In [ ]:
with timer("sub"):
    sub = pd.read_csv(SUB_PATH, header=None)
    sub[1] = y_test
    LOGGER.info(f'len sub={len(sub)}')
    sub.to_csv(os.path.join(SAVE_PATH,f'{EXP_ID}.csv'), index=False, header=None)

In [ ]:
"""importtances visualization"""
cols = importances[["feature", "gain"]].groupby("feature").mean().sort_values(
    by="gain", ascending=False)[:1000].index

best_features = importances.loc[importances.feature.isin(cols)]
plt.figure(figsize=(20,20))
sns.barplot(x="gain", y="feature", data=best_features.sort_values(by="gain", ascending=False))
plt.title(f'LightGBM Features (avg over folds)_{EXP_ID}')
plt.tight_layout()
plt.savefig(os.path.join(SAVE_PATH,(f'lgbm_importances_{EXP_ID}.png')))

In [ ]:
categorical_features
#TE_feature[TE_columns]

In [ ]:
pd.set_option('display.max_rows', 500)
importances.sort_values("gain", ascending=False)

In [ ]:
use_cols = list(importances[importances["gain"]>10000]["feature"].values)
len(use_cols)

In [ ]:
yoof_and_target[yoof_and_target["賃料"] == yoof_and_target["賃料"].max()]

In [ ]:
#yoof_and_target